In [38]:
import pandas as pd 
import boto3
import io
#s3_file_key = 'data/test.csv'
bucket_name = 'piyeanxia'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key='banking.csv')
initial_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
'''
or you can write like this 
initial_df = pd.read_csv("s3://nl12/banking.csv")
'''

'\nor you can write like this \ninitial_df = pd.read_csv("s3://nl12/banking.csv")\n'

In [39]:
#X = (initial_df.drop(['y'], axis=1)).fillna(0)
def transform_MultipleColumns_to_LabelEncoder_in_DataFrame(df):
    objList = df.select_dtypes(include = "object").columns #拿出為object的columns
    #print (objList)
    #Label Encoding for object to numeric conversion
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for feat in objList:
        df[feat] = le.fit_transform(df[feat].astype(str))  #在一個個做labelcndoer
    #print (X.info())
    return df

X = transform_MultipleColumns_to_LabelEncoder_in_DataFrame((initial_df.drop(['y'], axis=1)).fillna(0))
y = initial_df['y']
#split to train and test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [40]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(y_test.values, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.93      0.97      0.95     12034
     class 1       0.63      0.41      0.50      1559

    accuracy                           0.90     13593
   macro avg       0.78      0.69      0.72     13593
weighted avg       0.89      0.90      0.90     13593



In [42]:
new_result = pd.concat([X_test,y_test],axis=1)
new_result.insert((len(new_result.columns)), "predict", y_pred , True) 

In [43]:
location ="s3://"+bucket_name+"/new.csv"
new_result.to_csv(location, index=False)
print(type(location))

<class 'str'>
